# Querying Images using CLIP and YOLO

<div class="align-center">
  <a href="https://getindexify.ai/"><img src="https://getindexify.ai/Indexify_Logo_Wordmark.svg" width="145"></a>
  <a href="https://discord.com/invite/kF8UZACA7r"><img src="https://raw.githubusercontent.com/rishiraj/random/main/Discord%20button.png" width="145"></a><br>
  Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/tensorlakeai/indexify">Github</a></i> ⭐
</div>

This notebook demonstrates creating CLIP embeddings to search images based on a text query and YOLO object detection allowing you to query images containing specific objects.

In [1]:
from indexify import IndexifyClient, ExtractionGraph
client = IndexifyClient()

### Setup Extraction Graph
Create an Extraction Graph named visualknowledgebase with 2 extraction policies for clip and yolo extractors.

When an image is uploaded:

- The clipindex Extraction Policy will extract CLIP embeddings from this image and store it in clipindex.

- The objects Extraction Policy uses our YOLO Extractor and will add object metadata to the image content.

In [76]:
extraction_graph_spec = """
name: 'visualknowledgebase'
extraction_policies:
   - extractor: 'tensorlake/clip-extractor'
     name: 'clipindex'
   - extractor: 'tensorlake/yolo-extractor'
     name: 'objects'
"""

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)                                            

### Upload Files

Upload image files to our visualknowledgebase extraction graph for extraction.

In [77]:
file_names=["skate.jpg", "congestion.jpg", "bushwick-bred.jpg", "141900.jpg", "132500.jpg", "123801.jpg","120701.jpg", "103701.jpg"]
file_urls = [f"https://extractor-files.diptanu-6d5.workers.dev/images/{file_name}" for file_name in file_names]
for file_url in file_urls:
    cid = client.ingest_remote_file("visualknowledgebase", file_url, "image/png", {})
    client.wait_for_extraction(cid)

### View Indexes
View what indexes we can query

In [78]:
client.indexes()

[{'name': 'visualknowledgebase.clipindex.embedding',
  'embedding_schema': {'dim': 512, 'distance': 'cosine'}}]

### Query images with CLIP
Search the clip index to retrieve the most relevant results from your query. In this case we are searching for most relevant results for "skateboard"

In [79]:
client.search_index(name="visualknowledgebase.clipindex.embedding", query="skateboard", top_k=2)

[{'content_id': 'jfWdw1dsF8X80pxq',
  'text': '',
  'mime_type': 'image/png',
  'confidence_score': 0.7877066,
  'labels': {'bounding_box': '"[96.36837005615234, 73.69854736328125, 124.2857894897461, 106.82938385009766]"',
   'object_name': '"traffic light"'},
  'root_content_metadata': None,
  'content_metadata': {'id': 'jfWdw1dsF8X80pxq',
   'parent_id': '',
   'root_content_id': '',
   'namespace': 'default',
   'name': 'https://extractor-files.diptanu-6d5.workers.dev/images/skate.jpg',
   'mime_type': 'image/png',
   'labels': {},
   'extraction_graph_names': ['visualknowledgebase'],
   'storage_url': 'https://extractor-files.diptanu-6d5.workers.dev/images/skate.jpg',
   'created_at': 1715626710,
   'source': '',
   'size': 0,
   'hash': ''}},
 {'content_id': '5PEkB9nTk0SGTwfV',
  'text': '',
  'mime_type': 'image/png',
  'confidence_score': 0.81712943,
  'labels': {'object_name': '"\\"person\\""',
   'bounding_box': '"[497.01715087890625,308.41412353515625,527.5113525390625,469.82

### Query Structured Data with SQL
Indexify syncs the structured data with the vector data, so users can do semantic search on images with specific objects


In [82]:
client.sql_query("select * from visualknowledgebase where object_name='boat';")

SqlQueryResult(result=[{'bounding_box': [1536.345947265625, 1004.9735717773438, 1863.748779296875, 1330.083984375], 'content_id': 'yRR24K8T5FAsaFBO', 'object_name': 'boat'}, {'bounding_box': [203.15135192871097, 647.0704956054688, 635.392333984375, 1677.705810546875], 'content_id': 'yRR24K8T5FAsaFBO', 'object_name': 'boat'}, {'bounding_box': [599.879638671875, 277.8057861328125, 1368.181640625, 1384.4854736328125], 'content_id': 'yRR24K8T5FAsaFBO', 'object_name': 'boat'}, {'bounding_box': [99.08174896240234, 1202.60693359375, 164.80130004882812, 1330.62890625], 'content_id': 'yRR24K8T5FAsaFBO', 'object_name': 'boat'}, {'bounding_box': [2258.1826171875, 1236.341064453125, 2389.37353515625, 1328.902587890625], 'content_id': 'yRR24K8T5FAsaFBO', 'object_name': 'boat'}, {'bounding_box': [1410.8212890625, 1248.16259765625, 1551.63623046875, 1311.60888671875], 'content_id': 'yRR24K8T5FAsaFBO', 'object_name': 'boat'}, {'bounding_box': [601.3385009765625, 1207.789794921875, 1344.281982421875, 1